# import def func

In [29]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [30]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

from JSONData import tdx_hdf5_api as h5a
def get_tdx_all_from_h5(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d':
        dl='60'
    else:
        dl='200'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


def get_tdx_all_from_h5(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
    if end is not None:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
            '_' + end.replace('-', '') + '_' + 'all'
    else:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

## init data

### get_tdx_all

In [31]:
h5_day=get_tdx_all_from_h5(resample='d')

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


### sina

### sina_all

In [32]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-18',
 code
 600321    正源股份
 Name: name, dtype: object)

In [33]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/19/25 14:53:24'

code
300994   2025-06-19 14:52:21
301208   2025-06-19 14:52:21
Name: ticktime, dtype: datetime64[ns]

1.0555126031239828

#### 实时成交

In [34]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2507265, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-19 14:52:01,67.64,68.6,67.1,67.4,23735826,67.31
2025-06-19 14:52:19,67.62,68.6,67.1,67.4,23764726,67.31
2025-06-19 14:52:49,67.61,68.6,67.1,67.4,23806526,67.49


0.974

总量:9828.86 亿


ticktime    2025-06-20 00:00:00
close                 67.771677
volume                     0.17
vol                       23.81
amount                    16.13
Name: 603019, dtype: object

'2025-06-19'

#### 读取实时成交个股

In [35]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5416, 5),
         volume  name
 code                
 300308    0.81  中际旭创
 300468    0.56  四方精创
 300059    0.55  东方财富
 002463    0.52  沪电股份
 300502    0.50   新易盛
 002104    0.47  恒宝股份
 300773    0.46   拉卡拉
 002384    0.43  东山精密
 002015    0.39  协鑫能科
 002366    0.38  融发核电
 300100    0.38  双林股份
 300476    0.38  胜宏科技
 002050    0.38  三花智控
 300180    0.36  华峰超纤
 300750    0.36  宁德时代
 600519    0.35  贵州茅台
 002475    0.35  立讯精密
 300255    0.34  常山药业
 002594    0.34   比亚迪
 002017    0.33  东信和平)

##### 合并实时成交量及SinaDM

In [36]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### resample 3d

In [37]:
h5_3d = get_tdx_all_from_h5(resample='3d')
tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [38]:
tdx_3d.loc['000002'].close

6.32

In [39]:
tdx_3d.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')

show_tdx_data(tdx_select,limit=10)

(5182, 218)

Count:319 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000001,平安银行,2025-04-09,11.71,11.34,10.12,1.30,11.711880,10.12,0.10,1.3
000037,深南电A,2025-04-09,8.95,9.29,6.90,1.10,9.078675,6.90,21.09,-0.9
000096,广聚能源,2025-06-06,14.06,18.97,13.67,1.30,14.021503,13.67,0.00,-1.1
000403,派林生物,2025-05-15,18.22,19.02,15.98,1.10,18.208308,16.16,0.00,1.3
000408,藏格矿业,2025-04-09,40.74,37.66,31.00,1.20,40.936515,31.00,7.30,0.0
000518,*ST四环,2025-05-06,2.61,2.66,1.72,1.20,2.628700,1.81,33.41,12.9
000519,中兵红箭,2025-04-18,18.12,18.77,15.17,1.20,17.447431,16.30,0.00,0.2
000520,凤凰航运,2025-04-09,5.27,5.50,3.44,31.20,5.242170,3.44,0.10,0.0
000554,泰山石油,2025-04-09,8.08,6.39,5.00,52.31,7.674392,5.00,1.30,0.0


In [40]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
tdx_3d.shape
t3d_code = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
show_tdx_data(t3d_code)

(5182, 218)

Count:85 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000403,派林生物,2025-05-15,18.22,19.02,15.98,1.10,18.208308,16.16,0.00,1.3
000766,通化金马,2025-04-09,25.45,23.20,15.66,39.81,25.584678,15.66,0.00,20.2
002015,协鑫能科,2025-04-09,11.66,8.60,6.34,55.00,12.321957,6.34,1.20,1.2
002017,东信和平,2025-04-09,16.78,10.75,8.07,48.19,17.385945,8.07,28.00,12.3
002104,恒宝股份,2025-03-31,14.72,10.94,6.42,56.09,15.306525,6.42,12.10,26.0
002130,沃尔核材,2025-04-09,23.80,21.30,14.80,1.20,23.798763,14.80,1.20,0.1
002246,北化股份,2025-04-09,13.50,13.10,8.60,42.09,13.459170,8.60,1.20,17.2
002316,ST亚联,2025-04-09,4.87,4.44,3.09,28.91,4.962365,3.09,14.10,7.3
002365,永安药业,2025-03-31,25.44,22.39,8.18,1.30,25.587100,8.18,7.70,-1.1


### resample W

In [41]:
h5_w = get_tdx_all_from_h5(resample='w')
tdx_w = cct.combine_dataFrame(h5_w,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [42]:
tdx_w.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select_w = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# show_tdx_data(tdx_select_w)

(5186, 219)

In [43]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
show_tdx_data(tw_code,limit=10)

Count:1362 219


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
399001,NaN,2025-04-18,NaN,10992.00,9120.00,0.1,NaN,9648.00,1.3,-0.1
399006,NaN,2025-04-18,NaN,2282.00,1757.00,0.1,NaN,1880.00,1.3,-0.1
399005,NaN,2025-01-10,NaN,6860.00,5712.00,0.1,NaN,6012.00,1.2,0.0
999999,NaN,2025-01-10,NaN,3420.00,3040.00,0.1,NaN,3168.00,1.3,0.1
600156,NaN,2025-01-10,NaN,6.39,4.05,1.2,NaN,4.31,21.7,1.2
000001,平安银行,2025-04-11,11.71,11.61,10.12,1.3,11.711880,10.12,1.3,1.2
000020,深华发Ａ,2025-04-11,14.65,14.71,9.22,0.0,16.139640,9.22,1.1,1.2
000027,深圳能源,2025-01-10,6.32,6.57,5.64,0.1,6.307716,5.84,0.0,-0.1
000029,深深房Ａ,2025-01-10,19.43,20.02,12.81,0.0,19.724756,13.90,1.3,11.0


### resample M

In [44]:
h5_m = get_tdx_all_from_h5(resample='m')
tdx_m = cct.combine_dataFrame(h5_m,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [45]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
show_tdx_data(tm_code)

Count:1455 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
999999,NaN,2025-01-27,NaN,3352.00,2690.00,0.10,NaN,3140.00,-0.10,1.20
601028,NaN,2025-02-28,NaN,14.58,10.52,1.10,NaN,11.14,1.10,0.00
600156,NaN,2024-09-30,NaN,5.48,2.71,54.31,NaN,3.24,0.00,37.50
000001,平安银行,2025-04-30,11.71,11.60,9.00,1.30,11.711880,10.12,-1.10,1.20
000010,美丽生态,2024-09-30,2.96,3.03,1.38,30.00,3.008081,1.64,-1.10,1.10
000017,深中华A,2025-02-28,6.44,7.12,5.06,1.20,6.624234,5.50,0.00,0.10
000020,深华发Ａ,2025-04-30,14.65,15.72,9.00,1.30,16.139640,9.22,-1.10,-0.90
000027,深圳能源,2025-01-27,6.32,7.15,5.19,0.00,6.307716,5.80,0.00,0.10
000029,深深房Ａ,2025-01-27,19.43,18.05,10.00,1.30,19.724756,13.63,4.10,0.00


### resample D

In [46]:
h5_d = get_tdx_all_from_h5(resample='d')
tdx_d = cct.combine_dataFrame(h5_d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [47]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
show_tdx_data(td_code,limit=10)
if len(td_code) < 30:
    write_code_to_blk(td_code.index,'065',rewrite=True)

Count:44 220


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
688088,虹软科技,2025-04-07,45.28,47.31,42.41,0.20,45.521091,36.56,-0.10,1.2
836221,易实精密,2025-04-07,22.88,18.52,16.14,1.20,22.966083,11.11,-0.90,1.2
920088,科力股份,2025-04-07,60.23,31.66,25.09,54.19,56.247205,19.12,35.31,1.2
000731,四川美丰,2025-04-07,6.98,6.81,6.54,1.30,6.937011,6.41,0.00,0.1
002173,创新医疗,2025-04-08,13.59,12.12,9.33,30.30,13.555227,8.21,-1.10,-1.1
002207,准油股份,2025-04-08,10.05,6.24,5.50,32.41,10.049313,4.68,15.20,0.1
002348,高乐股份,2025-04-08,4.19,4.30,3.22,31.20,4.209597,2.80,-1.10,16.0
002371,北方华创,2025-06-12,423.58,457.50,402.00,1.30,425.820213,402.00,0.00,-0.1
002555,三七互娱,2025-04-07,16.51,15.06,14.19,1.20,16.443602,12.88,1.30,1.3


In [48]:
'002476' in td_code

False

In [49]:
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))

### check data

In [50]:
code_check = [co for co in code_f if co in tdx_3d.index]
len(code_check)

4

In [51]:
show_tdx_data(td_code.loc[code_f])

Count:4 220


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
920088,科力股份,2025-04-07,60.23,31.66,25.09,54.19,56.247205,19.12,35.31,1.2
300502,新易盛,2025-04-08,112.89,100.88,65.94,1.20,112.772654,48.56,0.10,-0.1
002826,易明医药,2025-04-08,23.88,23.73,10.26,27.30,24.111308,9.03,-0.90,17.3
002940,昂利康,2025-04-08,29.06,27.09,13.25,1.30,29.320359,12.12,-1.00,17.3


### Write toblk

In [52]:
# write_code_to_blk(code_f,'077',rewrite=False)